In [1]:
from datetime import timedelta, date, time, datetime
import time
import win32com.client
import pyodbc
import ctypes
import os
import pandas as pd
import numpy as np


start_dt = date(2021, 7, 1)
end_dt = date(2021, 9, 30)
Data = 'Stat'
Save_file = True


pyodbc.pooling = False

class DBSession:

    def __init__(self, dsn, pwd, str_conn=''):
        self.db_name = dsn
        self.pwd = pwd
        #        self.conn_str=';'.join(['DSN='+dsn, 'PWD='+pwd, 'CHARSET=UTF8'])
        self.conn_str = ';'.join(['DSN=' + dsn, 'CHARSET=UTF8'])
        if str_conn != '': self.conn_str = str_conn
        self.conn = None
        self.curs = None

    def open_conn(self):
        try:
            self.conn = pyodbc.connect(self.conn_str)
            self.conn.autocommit = True
            self.conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')

        except pyodbc.Error as ex:
            print("\nCan't connect to database " + self.db_name)
            print(ex)
            raise SystemExit(-1)

    def close_conn(self):
        if self is not None: self.conn.commit()
        if self is not None: self.conn.close()

    def exceptionHandler(self, ex, q, p):
        sqlstate = ex.args[0]
        if sqlstate == '42S02':  # Table does not exist (just inform in log)
            print(ex.args[1])
        else:
            print(q)
            print(p)
            raise ex

    def execute_query(self, query, params=None):
        try:
            self.curs = self.conn.cursor()
            if params:
                self.curs.execute(query, params)
            else:
                self.curs.execute(query)
        except pyodbc.Error as ex:
            self.exceptionHandler(ex, query, params)

        res = self.curs.fetchall()

        self.curs.close()

        return res
    
def collect_stat(start_dt, end_dt, opt = 'Stat', export = True):

    start_date, start_time, start_tm = date.today(), time.localtime(), datetime.now()
    
    print('Start at ', start_tm.strftime("%H:%M:%S"))
    
    
    def daterange(date1, date2):
        for n in range(int((date2 - date1).days) + 1):
            yield date1 + timedelta(n)
    
    
    date_list = []
    
    for dt in daterange(start_dt, end_dt):
        date_list.append(dt.strftime("%Y-%m-%d"))
    
    
    
    
    data_main, data_sup = [], []
    
    Global_Dev_DSN = "TDSB14"
    Global_Dev_Pass = "zaq1@WSXcde3"
    TeraDB = DBSession(Global_Dev_DSN, Global_Dev_Pass)
    TeraDB.open_conn()
    
    
    st = datetime.now()
    
    def dataCreate(Date_from, Date_to,  sqlQuery):
        SQL_DM_table = """
        select trim(databasename) as _user 
                , substr(t.commentstring, instr( t.commentstring, 'CAB'), 15) as _user_ad
        from dbc.databases t
        where t.databasename like any(  'PRD3_1_UDMOKFLEKP%'
                                                                            , 'prd3_1_UDMPLACC%'
                                                                            , 'PRD3_1_UDMPSSLCL%'
                                                                            , 'PRD3_1_UDMSBCERTS%'
                                                                           , 'prd3_1_UDMSLCL%'
                                                                            , 'PRD3_1_UDMSPCGBK%'
                                                                            , 'PRD3_1_UDMSPCPFL%'
                                                                            , 'PRD3_1_UDMSPCRFPL%'
                                                                            , 'PRD3_1_UDMTRANS%'
                                                                            , 'PRD3_Udwh_pr2%'
                                                                            , 'PRD3_UPR%'
                                                                            , 'PRD3_UPR2%'
        )
        and t.dbkind = 'U'
        and _user_ad like 'CA%'
        """
        SQLQuery_all_users = """
               select *
          from (
      
        select _user
          from (
    
                 select trim(databasename) as _user
        from dbc.databases t
        where t.databasename like any(
                                                                            'PRD3_LOAD_%'
                                                                           , 'PRD3_1_010_U%'
                                                                            , 'PRD3_1_015_U%'
                                                                            ,  'PRD3_1_016_U%'
                                                                            , 'PRD3_1_029_U%'
                                                                            , 'PRD3_1_037_U%'
                                                                            ,  'PRD3_1_068_U%'
                                                                            ,  'PRD3_1_069_U%'
                                                                            ,  'PRD3_1_076_U%'
                                                                            ,  'PRD3_1_083_U%'
                                                                            ,  'PRD3_1_200_U%'
                                                                            ,  'PRD3_1_207_U%'
                                                                            ,  'PRD3_1_210_U%'
                                                                            , 'PRD3_1_UDMOKFLEKP%'
                                                                            , 'prd3_1_UDMPLACC%'
                                                                            , 'PRD3_1_UDMPSSLCL%'
                                                                            , 'PRD3_1_UDMSBCERTS%'
                                                                           , 'prd3_1_UDMSLCL%'
                                                                            , 'PRD3_1_UDMSPCGBK%'
                                                                            , 'PRD3_1_UDMSPCPFL%'
                                                                            , 'PRD3_1_UDMSPCRFPL%'
                                                                            , 'PRD3_1_UDMTRANS%'
                                                                            , 'PRD3_Udwh_pr2%'
                                                                            , 'PRD3_UPR%'
                                                                            , 'PRD3_UPR2%'
                                                                            , 'PRD3_1_VD_DWH%'
                                                                            , 'PRD3_1_VD_TMD%'
                                                                                   , 'PRD3_1_DB_DWH%'
                                                                                   , 'PRD3_1_DB_STG%'
                                                                                   , 'PRD3_1_DB_STG_DDL%'
                                                                                   , 'PRD3_1_DB_TMD_ARCH%'
        )
        and t.dbkind = 'U'
        ) b
    
        union all
     
        select _user_ad
          from (
               select 
                 substr(t.commentstring, instr( t.commentstring, 'CAB'), 15) as _user_ad
        from dbc.databases t
        where t.databasename like any(
                                                                            'PRD3_LOAD_%'
                                                                           , 'PRD3_1_010_U%'
                                                                            , 'PRD3_1_015_U%'
                                                                            ,  'PRD3_1_016_U%'
                                                                            , 'PRD3_1_029_U%'
                                                                            , 'PRD3_1_037_U%'
                                                                            ,  'PRD3_1_068_U%'
                                                                            ,  'PRD3_1_069_U%'
                                                                            ,  'PRD3_1_076_U%'
                                                                            ,  'PRD3_1_083_U%'
                                                                            ,  'PRD3_1_200_U%'
                                                                            ,  'PRD3_1_207_U%'
                                                                            ,  'PRD3_1_210_U%'
                                                                            , 'PRD3_1_UDMOKFLEKP%'
                                                                            , 'prd3_1_UDMPLACC%'
                                                                            , 'PRD3_1_UDMPSSLCL%'
                                                                            , 'PRD3_1_UDMSBCERTS%'
                                                                           , 'prd3_1_UDMSLCL%'
                                                                            , 'PRD3_1_UDMSPCGBK%'
                                                                            , 'PRD3_1_UDMSPCPFL%'
                                                                            , 'PRD3_1_UDMSPCRFPL%'
                                                                            , 'PRD3_1_UDMTRANS%'
                                                                            , 'PRD3_Udwh_pr2%'
                                                                            , 'PRD3_UPR%'
                                                                            , 'PRD3_UPR2%'
                                                                            , 'PRD3_1_VD_DWH%'
                                                                            , 'PRD3_1_VD_TMD%'
                                                                                   , 'PRD3_1_DB_DWH%'
                                                                                   , 'PRD3_1_DB_STG%'
                                                                                   , 'PRD3_1_DB_STG_DDL%'
                                                                                   , 'PRD3_1_DB_TMD_ARCH%'
        )
        and t.dbkind = 'U'
        )  a
          where _user_ad like 'CA%'
           )  c
        """
        SQLQuery_DM_users = """
               select *
          from (
    
        select _user
          from (
    
                 select trim(databasename) as _user
               , case when  t.databasename like any(    'PRD3_1_UDMOKFLEKP%'
                                                                            , 'prd3_1_UDMPLACC%'
                                                                            , 'PRD3_1_UDMPSSLCL%'
                                                                            , 'PRD3_1_UDMSBCERTS%'
                                                                           , 'prd3_1_UDMSLCL%'
                                                                            , 'PRD3_1_UDMSPCGBK%'
                                                                            , 'PRD3_1_UDMSPCPFL%'
                                                                            , 'PRD3_1_UDMSPCRFPL%'
                                                                            , 'PRD3_1_UDMTRANS%'
                                                                            , 'PRD3_Udwh_pr2%'
                                                                            , 'PRD3_UPR%'
                                                                            , 'PRD3_UPR2%') then 'DataMart'
                        else 'Detail' end as _type
        from dbc.databases t
        where t.databasename like any(
                                                                            'PRD3_LOAD_%'
                                                                           , 'PRD3_1_010_U%'
                                                                            , 'PRD3_1_015_U%'
                                                                            ,  'PRD3_1_016_U%'
                                                                            , 'PRD3_1_029_U%'
                                                                            , 'PRD3_1_037_U%'
                                                                            ,  'PRD3_1_068_U%'
                                                                            ,  'PRD3_1_069_U%'
                                                                            ,  'PRD3_1_076_U%'
                                                                            ,  'PRD3_1_083_U%'
                                                                            ,  'PRD3_1_200_U%'
                                                                            ,  'PRD3_1_207_U%'
                                                                            ,  'PRD3_1_210_U%'
                                                                            , 'PRD3_1_UDMOKFLEKP%'
                                                                            , 'prd3_1_UDMPLACC%'
                                                                            , 'PRD3_1_UDMPSSLCL%'
                                                                            , 'PRD3_1_UDMSBCERTS%'
                                                                           , 'prd3_1_UDMSLCL%'
                                                                            , 'PRD3_1_UDMSPCGBK%'
                                                                            , 'PRD3_1_UDMSPCPFL%'
                                                                            , 'PRD3_1_UDMSPCRFPL%'
                                                                            , 'PRD3_1_UDMTRANS%'
                                                                            , 'PRD3_Udwh_pr2%'
                                                                            , 'PRD3_UPR%'
                                                                            , 'PRD3_UPR2%'
                                                                            , 'PRD3_1_VD_DWH%'
                                                                            , 'PRD3_1_VD_TMD%'
                                                                                   , 'PRD3_1_DB_DWH%'
                                                                                   , 'PRD3_1_DB_STG%'
                                                                                   , 'PRD3_1_DB_STG_DDL%'
                                                                                   , 'PRD3_1_DB_TMD_ARCH%'
        )
        and t.dbkind = 'U'
        ) b
          where  _type = 'DataMart'
    
    
    
    
        union all
    
     
        select _user_ad
          from (
               select 
                 substr(t.commentstring, instr( t.commentstring, 'CAB'), 15) as _user_ad
               , case when  t.databasename like any(    'PRD3_1_UDMOKFLEKP%'
                                                                            , 'prd3_1_UDMPLACC%'
                                                                            , 'PRD3_1_UDMPSSLCL%'
                                                                            , 'PRD3_1_UDMSBCERTS%'
                                                                           , 'prd3_1_UDMSLCL%'
                                                                            , 'PRD3_1_UDMSPCGBK%'
                                                                            , 'PRD3_1_UDMSPCPFL%'
                                                                            , 'PRD3_1_UDMSPCRFPL%'
                                                                            , 'PRD3_1_UDMTRANS%'
                                                                            , 'PRD3_Udwh_pr2%'
                                                                            , 'PRD3_UPR%'
                                                                            , 'PRD3_UPR2%') then 'DataMart'
                        else 'Detail' end as _type
        from dbc.databases t
        where t.databasename like any(
                                                                            'PRD3_LOAD_%'
                                                                           , 'PRD3_1_010_U%'
                                                                            , 'PRD3_1_015_U%'
                                                                            ,  'PRD3_1_016_U%'
                                                                            , 'PRD3_1_029_U%'
                                                                            , 'PRD3_1_037_U%'
                                                                            ,  'PRD3_1_068_U%'
                                                                            ,  'PRD3_1_069_U%'
                                                                            ,  'PRD3_1_076_U%'
                                                                            ,  'PRD3_1_083_U%'
                                                                            ,  'PRD3_1_200_U%'
                                                                            ,  'PRD3_1_207_U%'
                                                                            ,  'PRD3_1_210_U%'
                                                                            , 'PRD3_1_UDMOKFLEKP%'
                                                                            , 'prd3_1_UDMPLACC%'
                                                                            , 'PRD3_1_UDMPSSLCL%'
                                                                            , 'PRD3_1_UDMSBCERTS%'
                                                                           , 'prd3_1_UDMSLCL%'
                                                                            , 'PRD3_1_UDMSPCGBK%'
                                                                            , 'PRD3_1_UDMSPCPFL%'
                                                                            , 'PRD3_1_UDMSPCRFPL%'
                                                                            , 'PRD3_1_UDMTRANS%'
                                                                            , 'PRD3_Udwh_pr2%'
                                                                            , 'PRD3_UPR%'
                                                                            , 'PRD3_UPR2%'
                                                                            , 'PRD3_1_VD_DWH%'
                                                                            , 'PRD3_1_VD_TMD%'
                                                                                   , 'PRD3_1_DB_DWH%'
                                                                                   , 'PRD3_1_DB_STG%'
                                                                                   , 'PRD3_1_DB_STG_DDL%'
                                                                                   , 'PRD3_1_DB_TMD_ARCH%'
        )
        and t.dbkind = 'U'
        )  a
          where _user_ad like 'CA%'
          and _type = 'DataMart'
    
    
          )  c
    
            """
    
        if sqlQuery == 'Stat_MDM':
            SQLq = """ select 
            ql.LogDate,
            case when queryText like any(
            '%select%from%PRD3_1_010_DB_DWH%',
            '%select%from%PRD3_1_010_DB_STG%',
            '%select%from%PRD3_1_010_DB_TMD%',
            '%select%from%PRD3_1_010_VD_DWH%',
            '%select%from%PRD3_1_010_VD_TMD%',
            '%select%from%PRD3_1_015_DB_DWH%',
            '%select%from%PRD3_1_015_DB_STG%',
            '%select%from%PRD3_1_015_DB_TMD%',
            '%select%from%PRD3_1_015_VD_DWH%',
            '%select%from%PRD3_1_015_VD_TMD%',
            '%select%from%PRD3_1_016_DB_DWH%',
            '%select%from%PRD3_1_016_DB_STG%',
            '%select%from%PRD3_1_016_DB_TMD%',
            '%select%from%PRD3_1_016_VD_DWH%',
            '%select%from%PRD3_1_016_VD_TMD%',
            '%select%from%PRD3_1_029_DB_DWH%',
            '%select%from%PRD3_1_029_DB_STG%',
            '%select%from%PRD3_1_029_DB_TMD%',
            '%select%from%PRD3_1_029_VD_DWH%',
            '%select%from%PRD3_1_029_VD_TMD%',
            '%select%from%PRD3_1_037_DB_DWH%',
            '%select%from%PRD3_1_037_DB_STG%',
            '%select%from%PRD3_1_037_DB_TMD%',
            '%select%from%PRD3_1_037_VD_DWH%',
            '%select%from%PRD3_1_037_VD_TMD%',
            '%select%from%PRD3_1_068_DB_DWH%',
            '%select%from%PRD3_1_068_DB_STG%',
            '%select%from%PRD3_1_068_DB_TMD%',
            '%select%from%PRD3_1_068_VD_DWH%',
            '%select%from%PRD3_1_068_VD_TMD%',
            '%select%from%PRD3_1_069_DB_DWH%',
            '%select%from%PRD3_1_069_DB_STG%',
            '%select%from%PRD3_1_069_DB_TMD%',
            '%select%from%PRD3_1_069_VD_DWH%',
            '%select%from%PRD3_1_069_VD_TMD%',
            '%select%from%PRD3_1_076_DB_DWH%',
            '%select%from%PRD3_1_076_DB_STG%',
            '%select%from%PRD3_1_076_DB_TMD%',
            '%select%from%PRD3_1_076_VD_DWH%',
            '%select%from%PRD3_1_076_VD_TMD%',
            '%select%from%PRD3_1_083_DB_DWH%',
            '%select%from%PRD3_1_083_DB_STG%',
            '%select%from%PRD3_1_083_DB_TMD%',
            '%select%from%PRD3_1_083_VD_DWH%',
            '%select%from%PRD3_1_083_VD_TMD%',
            '%select%from%prd3_1_200_DB_DWH%',
            '%select%from%prd3_1_200_DB_STG%',
            '%select%from%prd3_1_200_DB_TMD%',
            '%select%from%prd3_1_200_VD_DWH%',
            '%select%from%prd3_1_200_VD_TMD%',
            '%select%from%PRD3_1_207_DB_DWH%',
            '%select%from%PRD3_1_207_DB_STG%',
            '%select%from%PRD3_1_207_DB_TMD%',
            '%select%from%PRD3_1_207_VD_DWH%',
            '%select%from%PRD3_1_207_VD_TMD%',
            '%select%from%PRD3_1_210_DB_DWH%',
            '%select%from%PRD3_1_210_DB_STG%',
            '%select%from%PRD3_1_210_DB_TMD%',
            '%select%from%PRD3_1_210_VD_DWH%',
            '%select%from%PRD3_1_210_VD_TMD%',
            '%select%from%PRD3_1_DB_DWH%',
            '%select%from%PRD3_1_DB_STG%',
            '%select%from%PRD3_1_DB_STG_DDL%',
            '%select%from%PRD3_1_DB_TMD_ARCH%',
            '%select%from%PRD3_1_VD_DWH%',
            '%select%from%PRD3_1_VD_TMD%',
            '%select%from%PRD3_DB_DWH%',
            '%select%from%PRD3_DB_STG%',
            '%select%from%PRD3_DB_STG_DDL%',
            '%select%from%PRD3_VD_DWH%',
            '%select%from%PRD3_VD_TMD%')
            then 'DetailData'
            else 'Data Mart'
            end as data_type,
            case 
            when  queryText like any(
            '%select%from%PRD3_1_010_DB_DWH%',
            '%select%from%PRD3_1_010_DB_STG%',
            '%select%from%PRD3_1_010_DB_TMD%',
            '%select%from%PRD3_1_010_VD_DWH%',
            '%select%from%PRD3_1_010_VD_TMD%') then 'ЕКС'
        
            when  queryText like any(
            '%select%from%PRD3_1_015_DB_DWH%',
            '%select%from%PRD3_1_015_DB_STG%',
            '%select%from%PRD3_1_015_DB_TMD%',
            '%select%from%PRD3_1_015_VD_DWH%',
            '%select%from%PRD3_1_015_VD_TMD%') then 'Инфобанк'
        
        
            when  queryText like any(
            '%select%from%PRD3_1_016_DB_DWH%',
            '%select%from%PRD3_1_016_DB_STG%',
            '%select%from%PRD3_1_016_DB_TMD%',
            '%select%from%PRD3_1_016_VD_DWH%',
            '%select%from%PRD3_1_016_VD_TMD%') then 'Way4'
        
            when  queryText like any(
            '%select%from%PRD3_1_029_DB_DWH%',
            '%select%from%PRD3_1_029_DB_STG%',
            '%select%from%PRD3_1_029_DB_TMD%',
            '%select%from%PRD3_1_029_VD_DWH%',
            '%select%from%PRD3_1_029_VD_TMD%') then 'УВДО'
        
            when  queryText like any(
            '%select%from%PRD3_1_037_DB_DWH%',
            '%select%from%PRD3_1_037_DB_STG%',
            '%select%from%PRD3_1_037_DB_TMD%',
            '%select%from%PRD3_1_037_VD_DWH%',
            '%select%from%PRD3_1_037_VD_TMD%') then 'Transact Retail'
        
            when  queryText like any(
            '%select%from%PRD3_1_068_DB_DWH%',
            '%select%from%PRD3_1_068_DB_STG%',
            '%select%from%PRD3_1_068_DB_TMD%',
            '%select%from%PRD3_1_068_VD_DWH%',
            '%select%from%PRD3_1_068_VD_TMD%') then 'Статотчетность'
        
            when  queryText like any(
            '%select%from%PRD3_1_069_DB_DWH%',
            '%select%from%PRD3_1_069_DB_STG%',
            '%select%from%PRD3_1_069_DB_TMD%',
            '%select%from%PRD3_1_069_VD_DWH%',
            '%select%from%PRD3_1_069_VD_TMD%') then 'ЕКП'
        
            when  queryText like any(
            '%select%from%PRD3_1_076_DB_DWH%',
            '%select%from%PRD3_1_076_DB_STG%',
            '%select%from%PRD3_1_076_DB_TMD%',
            '%select%from%PRD3_1_076_VD_DWH%',
            '%select%from%PRD3_1_076_VD_TMD%') then 'Transact SME'
        
            when  queryText like any(
            '%select%from%PRD3_1_083_DB_DWH%',
            '%select%from%PRD3_1_083_DB_STG%',
            '%select%from%PRD3_1_083_DB_TMD%',
            '%select%from%PRD3_1_083_VD_DWH%',
            '%select%from%PRD3_1_083_VD_TMD%') then 'Юпитер'
        
            when  queryText like any(
            '%select%from%PRD3_1_100_DB_DWH%',
            '%select%from%PRD3_1_100_DB_STG%',
            '%select%from%PRD3_1_100_DB_TMD%',
            '%select%from%PRD3_1_100_VD_DWH%',
            '%select%from%PRD3_1_100_VD_TMD%') then 'Diasoft Dealing'
        
            when  queryText like any(
            '%select%from%PRD3_1_101_DB_DWH%',
            '%select%from%PRD3_1_101_DB_STG%',
            '%select%from%PRD3_1_101_DB_TMD%',
            '%select%from%PRD3_1_101_VD_DWH%',
            '%select%from%PRD3_1_101_VD_TMD%') then 'Diasoft Custody'
        
            when  queryText like any(
            '%select%from%prd3_1_200_DB_DWH%',
            '%select%from%prd3_1_200_DB_STG%',
            '%select%from%prd3_1_200_DB_TMD%',
            '%select%from%prd3_1_200_VD_DWH%',
            '%select%from%prd3_1_200_VD_TMD%') then 'ППРБ'
        
            when  queryText like any(
            '%select%from%PRD3_1_210_DB_DWH%',
            '%select%from%PRD3_1_210_DB_STG%',
            '%select%from%PRD3_1_210_DB_TMD%',
            '%select%from%PRD3_1_210_VD_DWH%',
            '%select%from%PRD3_1_210_VD_TMD%') then 'ППРБ.ТК'
        
            when  queryText like any(
            '%select%from%PRD3_1_DB_DWH%',
            '%select%from%PRD3_1_DB_STG%',
            '%select%from%PRD3_1_DB_STG_DDL%',
            '%select%from%PRD3_1_DB_TMD_ARCH%',
            '%select%from%PRD3_1_VD_DWH%',
            '%select%from%PRD3_1_VD_TMD%') then 'CRM Корп'
        
            when  queryText like any(
            '%select%from%PRD3_DB_DWH%',
            '%select%from%PRD3_DB_STG%',
            '%select%from%PRD3_DB_STG_DDL%',
            '%select%from%PRD3_VD_DWH%',
            '%select%from%PRD3_VD_TMD%') then 'ЦОД, СВО, Г2БК, АЕ ГР'
        
            when  queryText like any(
            '%select%from%PRD3_1_DB_DMOKFLEKP%', '%select%from%PRD3_1_VD_DMOKFLEKP%') then 'ВД КФЛ'
        
            when  queryText like any(
            '%select%from%prd3_1_DB_DMPLACC%','%select%from%PRD3_1_VD_DMPLACC%') then 'ВД PLACC'
        
            when  queryText like any(
            '%select%from%PRD3_1_VD_DMPSSLCL%','%select%from%PRD3_1_DB_DMPSSLCL%') then 'ВД Проверки КПЮЛ'
        
            when  queryText like any(
            '%select%from%PRD3_1_VD_DMSBCERTS%','%select%from%PRD3_1_DB_DMSBCERTS%') then 'ВД СберСертификаты'
        
            when  queryText like any(
            '%select%from%prd3_1_VD_DMSLCL%','%select%from%prd3_1_DB_DMSLCL%') then 'ВД КПЮЛ'
        
            when  queryText like any(
            '%select%from%PRD3_1_VD_DMSPCGBK%','%select%from%PRD3_1_DB_DMSPCGBK%') then 'ВД ГБК'
        
            when  queryText like any(
            '%select%from%PRD3_1_VD_DMSPCPFL%','%select%from%PRD3_1_DB_DMSPCPFL%') then 'ВД ВФЛ'
        
            when  queryText like any(
            '%select%from%PRD3_1_VD_DMSPCRFPL%','%select%from%PRD3_1_DB_DMSPCRFPL%') then 'ВД НПЮЛ'
        
            when  queryText like any(
            '%select%from%PRD3_1_VD_DMTRANS%','%select%from%PRD3_1_DB_DMTRANS%') then 'ВД Переводы'
        
        
            when  queryText like any('%select%from%PRD3_vd_dwh_pr2%',
            '%select%from%PRD3_VD_PR%',
            '%select%from%PRD3_VD_PR2%') then 'ВД ТБО'
        
            else 'Othen'
            end as detail_,
            UserName,
                     case 
                               when  coalesce(users._user, UserName) like 'PRD3_1_UDMOKFLEKP%' then 'ВД КФЛ'
        
            when  coalesce(users._user, UserName) like 'prd3_1_UDMPLACC%' then 'ВД PLACC'
        
        
            when   coalesce(users._user, UserName) like 'PRD3_1_UDMSBCERTS%' then 'ВД СберСертификаты'
        
            when   coalesce(users._user, UserName) like 'prd3_1_UDMSLCL%' then 'ВД КПЮЛ'
        
            when   coalesce(users._user, UserName) like 'PRD3_1_UDMSPCGBK%' then 'ВД ГБК'
        
            when   coalesce(users._user, UserName) like 'PRD3_1_UDMSPCPFL%' then 'ВД ВФЛ'
        
            when   coalesce(users._user, UserName) like 'PRD3_1_UDMSPCRFPL%' then 'ВД НПЮЛ'
        
            when   coalesce(users._user, UserName) like 'PRD3_1_UDMTRANS%' then 'ВД Переводы'
        
        
            when   coalesce(users._user, UserName) like any('PRD3_Udwh_pr2%'
                                                                    , 'PRD3_UPR%'
                                                                    , 'PRD3_UPR2%') then 'ВД ТБО'
        
                else 'other'
                end DM,
            sum(cast((ReqPhysIOKB * MaxAmpIO * (HASHAMP()+ 1 )) / NULLIFZERO(TotalIOCount) as float)) ImpactDisk,
            sum(ReqPhysIOKB) as ReqPhysIOKB,
            sum(AMPCPUTime) as AMPCPUTime,
            sum(TotalIOCount) as TotalIOCount,
            sum(cast(ql.MaxAMPCPUTime * (ql.NumOfActiveAMPs) as float)) AS ImpactCPU,
            count(*) as sql_cnt
            from 
            pdcrinfo.DBQLogTbl_Hst ql 
            left join ( """+ SQL_DM_table + """) users
              on users._user_ad =  ql.UserName
            where
            ql.LogDate between date'""" + Date_from + """' and date'""" + Date_to + """' 
            and queryText like any(
            '%select%from%PRD3_1_010_DB_DWH%',
            '%select%from%PRD3_1_010_DB_STG%',
            '%select%from%PRD3_1_010_DB_TMD%',
            '%select%from%PRD3_1_010_VD_DWH%',
            '%select%from%PRD3_1_010_VD_TMD%',
            '%select%from%PRD3_1_015_DB_DWH%',
            '%select%from%PRD3_1_015_DB_STG%',
            '%select%from%PRD3_1_015_DB_TMD%',
            '%select%from%PRD3_1_015_VD_DWH%',
            '%select%from%PRD3_1_015_VD_TMD%',
            '%select%from%PRD3_1_016_DB_DWH%',
            '%select%from%PRD3_1_016_DB_STG%',
            '%select%from%PRD3_1_016_DB_TMD%',
            '%select%from%PRD3_1_016_VD_DWH%',
            '%select%from%PRD3_1_016_VD_TMD%',
            '%select%from%PRD3_1_029_DB_DWH%',
            '%select%from%PRD3_1_029_DB_STG%',
            '%select%from%PRD3_1_029_DB_TMD%',
            '%select%from%PRD3_1_029_VD_DWH%',
            '%select%from%PRD3_1_029_VD_TMD%',
            '%select%from%PRD3_1_037_DB_DWH%',
            '%select%from%PRD3_1_037_DB_STG%',
            '%select%from%PRD3_1_037_DB_TMD%',
            '%select%from%PRD3_1_037_VD_DWH%',
            '%select%from%PRD3_1_037_VD_TMD%',
            '%select%from%PRD3_1_068_DB_DWH%',
            '%select%from%PRD3_1_068_DB_STG%',
            '%select%from%PRD3_1_068_DB_TMD%',
            '%select%from%PRD3_1_068_VD_DWH%',
            '%select%from%PRD3_1_068_VD_TMD%',
            '%select%from%PRD3_1_069_DB_DWH%',
            '%select%from%PRD3_1_069_DB_STG%',
            '%select%from%PRD3_1_069_DB_TMD%',
            '%select%from%PRD3_1_069_VD_DWH%',
            '%select%from%PRD3_1_069_VD_TMD%',
            '%select%from%PRD3_1_076_DB_DWH%',
            '%select%from%PRD3_1_076_DB_STG%',
            '%select%from%PRD3_1_076_DB_TMD%',
            '%select%from%PRD3_1_076_VD_DWH%',
            '%select%from%PRD3_1_076_VD_TMD%',
            '%select%from%PRD3_1_083_DB_DWH%',
            '%select%from%PRD3_1_083_DB_STG%',
            '%select%from%PRD3_1_083_DB_TMD%',
            '%select%from%PRD3_1_083_VD_DWH%',
            '%select%from%PRD3_1_083_VD_TMD%',
        
            '%select%from%PRD3_1_100_DB_DWH%',
            '%select%from%PRD3_1_100_DB_STG%',
            '%select%from%PRD3_1_100_DB_TMD%',
            '%select%from%PRD3_1_100_VD_DWH%',
            '%select%from%PRD3_1_100_VD_TMD%',
        
            '%select%from%PRD3_1_101_DB_DWH%',
            '%select%from%PRD3_1_101_DB_STG%',
            '%select%from%PRD3_1_101_DB_TMD%',
            '%select%from%PRD3_1_101_VD_DWH%',
            '%select%from%PRD3_1_101_VD_TMD%',
        
            '%select%from%prd3_1_200_DB_DWH%',
            '%select%from%prd3_1_200_DB_STG%',
            '%select%from%prd3_1_200_DB_TMD%',
            '%select%from%prd3_1_200_VD_DWH%',
            '%select%from%prd3_1_200_VD_TMD%',
            '%select%from%PRD3_1_207_DB_DWH%',
            '%select%from%PRD3_1_207_DB_STG%',
            '%select%from%PRD3_1_207_DB_TMD%',
            '%select%from%PRD3_1_207_VD_DWH%',
            '%select%from%PRD3_1_207_VD_TMD%',
            '%select%from%PRD3_1_210_DB_DWH%',
            '%select%from%PRD3_1_210_DB_STG%',
            '%select%from%PRD3_1_210_DB_TMD%',
            '%select%from%PRD3_1_210_VD_DWH%',
            '%select%from%PRD3_1_210_VD_TMD%',
            '%select%from%PRD3_1_DB_DWH%',
            '%select%from%PRD3_1_DB_STG%',
            '%select%from%PRD3_1_DB_STG_DDL%',
            '%select%from%PRD3_1_DB_TMD_ARCH%',
            '%select%from%PRD3_1_VD_DWH%',
            '%select%from%PRD3_1_VD_TMD%',
            '%select%from%PRD3_DB_DWH%',
            '%select%from%PRD3_DB_STG%',
            '%select%from%PRD3_DB_STG_DDL%',
            '%select%from%PRD3_VD_DWH%',
            '%select%from%PRD3_VD_TMD%')
            and statementType in ('select','insert','create table', 'update', 'merge into')
            
            group by 1,2,3,4,5
            """
        elif sqlQuery == 'Stat':
            SQLq = """ select 
                ql.LogDate, 
                case when queryText like any(
                '%select%from%PRD3_1_010_DB_DWH%',
                '%select%from%PRD3_1_010_DB_STG%',
                '%select%from%PRD3_1_010_DB_TMD%',
                '%select%from%PRD3_1_010_VD_DWH%',
                '%select%from%PRD3_1_010_VD_TMD%',
                '%select%from%PRD3_1_015_DB_DWH%',
                '%select%from%PRD3_1_015_DB_STG%',
                '%select%from%PRD3_1_015_DB_TMD%',
                '%select%from%PRD3_1_015_VD_DWH%',
                '%select%from%PRD3_1_015_VD_TMD%',
                '%select%from%PRD3_1_016_DB_DWH%',
                '%select%from%PRD3_1_016_DB_STG%',
                '%select%from%PRD3_1_016_DB_TMD%',
                '%select%from%PRD3_1_016_VD_DWH%',
                '%select%from%PRD3_1_016_VD_TMD%',
                '%select%from%PRD3_1_029_DB_DWH%',
                '%select%from%PRD3_1_029_DB_STG%',
                '%select%from%PRD3_1_029_DB_TMD%',
                '%select%from%PRD3_1_029_VD_DWH%',
                '%select%from%PRD3_1_029_VD_TMD%',
                '%select%from%PRD3_1_037_DB_DWH%',
                '%select%from%PRD3_1_037_DB_STG%',
                '%select%from%PRD3_1_037_DB_TMD%',
                '%select%from%PRD3_1_037_VD_DWH%',
                '%select%from%PRD3_1_037_VD_TMD%',
                '%select%from%PRD3_1_068_DB_DWH%',
                '%select%from%PRD3_1_068_DB_STG%',
                '%select%from%PRD3_1_068_DB_TMD%',
                '%select%from%PRD3_1_068_VD_DWH%',
                '%select%from%PRD3_1_068_VD_TMD%',
                '%select%from%PRD3_1_069_DB_DWH%',
                '%select%from%PRD3_1_069_DB_STG%',
                '%select%from%PRD3_1_069_DB_TMD%',
                '%select%from%PRD3_1_069_VD_DWH%',
                '%select%from%PRD3_1_069_VD_TMD%',
                '%select%from%PRD3_1_076_DB_DWH%',
                '%select%from%PRD3_1_076_DB_STG%',
                '%select%from%PRD3_1_076_DB_TMD%',
                '%select%from%PRD3_1_076_VD_DWH%',
                '%select%from%PRD3_1_076_VD_TMD%',
                '%select%from%PRD3_1_083_DB_DWH%',
                '%select%from%PRD3_1_083_DB_STG%',
                '%select%from%PRD3_1_083_DB_TMD%',
                '%select%from%PRD3_1_083_VD_DWH%',
                '%select%from%PRD3_1_083_VD_TMD%',
                '%select%from%prd3_1_200_DB_DWH%',
                '%select%from%prd3_1_200_DB_STG%',
                '%select%from%prd3_1_200_DB_TMD%',
                '%select%from%prd3_1_200_VD_DWH%',
                '%select%from%prd3_1_200_VD_TMD%',
                '%select%from%PRD3_1_207_DB_DWH%',
                '%select%from%PRD3_1_207_DB_STG%',
                '%select%from%PRD3_1_207_DB_TMD%',
                '%select%from%PRD3_1_207_VD_DWH%',
                '%select%from%PRD3_1_207_VD_TMD%',
                '%select%from%PRD3_1_210_DB_DWH%',
                '%select%from%PRD3_1_210_DB_STG%',
                '%select%from%PRD3_1_210_DB_TMD%',
                '%select%from%PRD3_1_210_VD_DWH%',
                '%select%from%PRD3_1_210_VD_TMD%',
                '%select%from%PRD3_1_DB_DWH%',
                '%select%from%PRD3_1_DB_STG%',
                '%select%from%PRD3_1_DB_STG_DDL%',
                '%select%from%PRD3_1_DB_TMD_ARCH%',
                '%select%from%PRD3_1_VD_DWH%',
                '%select%from%PRD3_1_VD_TMD%',
                '%select%from%PRD3_DB_DWH%',
                '%select%from%PRD3_DB_STG%',
                '%select%from%PRD3_DB_STG_DDL%',
                '%select%from%PRD3_VD_DWH%',
                '%select%from%PRD3_VD_TMD%')
                then 'DetailData'
                else 'Data Mart'
                end as data_type,
                case 
                when  queryText like any(
                '%select%from%PRD3_1_010_DB_DWH%',
                '%select%from%PRD3_1_010_DB_STG%',
                '%select%from%PRD3_1_010_DB_TMD%',
                '%select%from%PRD3_1_010_VD_DWH%',
                '%select%from%PRD3_1_010_VD_TMD%') then 'ЕКС'
    
                when  queryText like any(
                '%select%from%PRD3_1_015_DB_DWH%',
                '%select%from%PRD3_1_015_DB_STG%',
                '%select%from%PRD3_1_015_DB_TMD%',
                '%select%from%PRD3_1_015_VD_DWH%',
                '%select%from%PRD3_1_015_VD_TMD%') then 'Инфобанк'
    
    
                when  queryText like any(
                '%select%from%PRD3_1_016_DB_DWH%',
                '%select%from%PRD3_1_016_DB_STG%',
                '%select%from%PRD3_1_016_DB_TMD%',
                '%select%from%PRD3_1_016_VD_DWH%',
                '%select%from%PRD3_1_016_VD_TMD%') then 'Way4'
    
                when  queryText like any(
                '%select%from%PRD3_1_029_DB_DWH%',
                '%select%from%PRD3_1_029_DB_STG%',
                '%select%from%PRD3_1_029_DB_TMD%',
                '%select%from%PRD3_1_029_VD_DWH%',
                '%select%from%PRD3_1_029_VD_TMD%') then 'УВДО'
    
                when  queryText like any(
                '%select%from%PRD3_1_037_DB_DWH%',
                '%select%from%PRD3_1_037_DB_STG%',
                '%select%from%PRD3_1_037_DB_TMD%',
                '%select%from%PRD3_1_037_VD_DWH%',
                '%select%from%PRD3_1_037_VD_TMD%') then 'Transact Retail'
    
                when  queryText like any(
                '%select%from%PRD3_1_068_DB_DWH%',
                '%select%from%PRD3_1_068_DB_STG%',
                '%select%from%PRD3_1_068_DB_TMD%',
                '%select%from%PRD3_1_068_VD_DWH%',
                '%select%from%PRD3_1_068_VD_TMD%') then 'Статотчетность'
    
                when  queryText like any(
                '%select%from%PRD3_1_069_DB_DWH%',
                '%select%from%PRD3_1_069_DB_STG%',
                '%select%from%PRD3_1_069_DB_TMD%',
                '%select%from%PRD3_1_069_VD_DWH%',
                '%select%from%PRD3_1_069_VD_TMD%') then 'ЕКП'
    
                when  queryText like any(
                '%select%from%PRD3_1_076_DB_DWH%',
                '%select%from%PRD3_1_076_DB_STG%',
                '%select%from%PRD3_1_076_DB_TMD%',
                '%select%from%PRD3_1_076_VD_DWH%',
                '%select%from%PRD3_1_076_VD_TMD%') then 'Transact SME'
    
                when  queryText like any(
                '%select%from%PRD3_1_083_DB_DWH%',
                '%select%from%PRD3_1_083_DB_STG%',
                '%select%from%PRD3_1_083_DB_TMD%',
                '%select%from%PRD3_1_083_VD_DWH%',
                '%select%from%PRD3_1_083_VD_TMD%') then 'Юпитер'
    
                when  queryText like any(
                '%select%from%prd3_1_200_DB_DWH%',
                '%select%from%prd3_1_200_DB_STG%',
                '%select%from%prd3_1_200_DB_TMD%',
                '%select%from%prd3_1_200_VD_DWH%',
                '%select%from%prd3_1_200_VD_TMD%') then 'ППРБ'
    
                when  queryText like any(
                '%select%from%PRD3_1_210_DB_DWH%',
                '%select%from%PRD3_1_210_DB_STG%',
                '%select%from%PRD3_1_210_DB_TMD%',
                '%select%from%PRD3_1_210_VD_DWH%',
                '%select%from%PRD3_1_210_VD_TMD%') then 'ППРБ.ТК'
    
                when  queryText like any(
                '%select%from%PRD3_1_DB_DWH%',
                '%select%from%PRD3_1_DB_STG%',
                '%select%from%PRD3_1_DB_STG_DDL%',
                '%select%from%PRD3_1_DB_TMD_ARCH%',
                '%select%from%PRD3_1_VD_DWH%',
                '%select%from%PRD3_1_VD_TMD%') then 'CRM Корп'
    
                when  queryText like any(
                '%select%from%PRD3_DB_DWH%',
                '%select%from%PRD3_DB_STG%',
                '%select%from%PRD3_DB_STG_DDL%',
                '%select%from%PRD3_VD_DWH%',
                '%select%from%PRD3_VD_TMD%') then 'ЦОД, СВО, Г2БК, АЕ ГР'
    
                when  queryText like any(
                '%select%from%PRD3_1_DB_DMOKFLEKP%', '%select%from%PRD3_1_VD_DMOKFLEKP%') then 'ВД КФЛ'
    
                when  queryText like any(
                '%select%from%prd3_1_DB_DMPLACC%','%select%from%PRD3_1_VD_DMPLACC%') then 'ВД PLACC'
    
                when  queryText like any(
                '%select%from%PRD3_1_VD_DMPSSLCL%','%select%from%PRD3_1_DB_DMPSSLCL%') then 'ВД Проверки КПЮЛ'
    
                when  queryText like any(
                '%select%from%PRD3_1_VD_DMSBCERTS%','%select%from%PRD3_1_DB_DMSBCERTS%') then 'ВД СберСертификаты'
    
                when  queryText like any(
                '%select%from%prd3_1_VD_DMSLCL%','%select%from%prd3_1_DB_DMSLCL%') then 'ВД КПЮЛ'
    
                when  queryText like any(
                '%select%from%PRD3_1_VD_DMSPCGBK%','%select%from%PRD3_1_DB_DMSPCGBK%') then 'ВД ГБК'
    
                when  queryText like any(
                '%select%from%PRD3_1_VD_DMSPCPFL%','%select%from%PRD3_1_DB_DMSPCPFL%') then 'ВД ВФЛ'
    
                when  queryText like any(
                '%select%from%PRD3_1_VD_DMSPCRFPL%','%select%from%PRD3_1_DB_DMSPCRFPL%') then 'ВД НПЮЛ'
    
                when  queryText like any(
                '%select%from%PRD3_1_VD_DMTRANS%','%select%from%PRD3_1_DB_DMTRANS%') then 'ВД Переводы'
    
    
                when  queryText like any('%select%from%PRD3_vd_dwh_pr2%',
                '%select%from%PRD3_VD_PR%',
                '%select%from%PRD3_VD_PR2%') then 'ВД ТБО'
    
               -- else 'Othen'
                end as detail_,
                  coalesce(ProxyUser, UserName) as _UserName,
                         coalesce(case 
                                 -- WHEN ql.FinalWDName LIKE 'WD-LOAD1%'  THEN 'LOAD1'
                                 -- WHEN ql.FinalWDName LIKE 'WD-LOAD2%'  THEN 'LOAD2'
                                  WHEN ql.FinalWDName LIKE 'WD-Fin%'         THEN 'Fin'
                                  WHEN ql.FinalWDName LIKE 'WD-RB%'            THEN 'RB'
                                  WHEN ql.FinalWDName LIKE 'WD-Retail%'            THEN 'RB'
                                 -- WHEN ql.FinalWDName LIKE 'WD-TECH%'    THEN 'TECH'
                                  WHEN ql.FinalWDName LIKE 'WD-CIB%'        THEN 'CIB'
                                  WHEN ql.FinalWDName LIKE 'WD-CORP%'        THEN 'CIB'
                                  WHEN ql.FinalWDName LIKE 'WD-SN%'         THEN 'SN'
                                  WHEN ql.FinalWDName LIKE 'WD-SALESNET%'         THEN 'SN'
                                  WHEN ql.FinalWDName LIKE 'WD-PRPA%'    THEN 'PRPA'
                                  WHEN ql.FinalWDName LIKE 'WD-RISK%' 
                                  OR        ql.FinalWDName LIKE 'WD-TD2OD%' THEN 'RISK'
                                  WHEN ql.FinalWDName LIKE 'WD-HR%'        THEN 'HR'
                                  WHEN ql.FinalWDName LIKE 'WD-UB%'        THEN 'UB'
                                  --WHEN ql.FinalWDName LIKE '%Utility%'        THEN 'Utility'
                                  --WHEN ql.FinalWDName LIKE '%QG%'        THEN 'QueryGrid'
                    -- else 'other'
                    end, dd.BB) BB,
                sum(cast((ReqPhysIOKB * MaxAmpIO * (HASHAMP()+ 1 )) / NULLIFZERO(TotalIOCount) as float)) ImpactDisk,
                sum(ReqPhysIOKB) as ReqPhysIOKB,
                sum(AMPCPUTime) as AMPCPUTime,
                sum(TotalIOCount) as TotalIOCount,
                sum(cast(ql.MaxAMPCPUTime * (ql.NumOfActiveAMPs) as float)) AS ImpactCPU,
                count(*) as sql_cnt
                from 
                pdcrinfo.DBQLogTbl_Hst ql 
                left join 
	
	(
	select databasename
        , ownername
		 
		  , CASE 
		     when ownername like '%BLAGO%' then 'UB'
			 when ownername like '%FIN%' 
			     OR databasename like any('%FIN%',  '%FKSMA%', '%FMALA%', '%FMCCA%', '%FMCOA%'
				                                            , '%FMCRA%', '%FMFCA%', '%FMAS0%', '%FMASA%', '%FMCF0%'
															, '%FMCIA%', '%FMKOA%', '%FMPR0%', '%FMPRA%', '%FMRCA%'
															, '%FMSC0%', '%FMSC0%', '%FMCFA%', '%FMSCA%', '%FMUV0%'
															, '%FMUVA%', '%FMXX0%', '%FMXXA%', '%FUHMA%') 
				 then 'Fin'
			 when ownername like '%ROZN%' then 'RB'
			 when ownername like '%SALESNTWRK%' then 'SN'
			 when ownername like '%STRATEGY%' then 'STRATEGY'
			 when ownername like '%RISK%' or databasename like '%RSK00%' then 'RISK'
			 when ownername like '%CIB%' or databasename like '%CIB%' then 'CIB'
			 when ownername like '%DRPA%' or databasename like '%DRPA%'  then 'PRPA'
			 when ownername like '%HR%' then 'HR'
			 
			end as BB
		 -- , commentstring
  from DBC.Databases
  where databasename like 'CAB-SA%'
  and databasename not like 'CAB-SA-TDT%'
	) as dd
	  on dd.DatabaseName = _UserName
                where
                ql.LogDate between date'""" + Date_from + """' and date'""" + Date_to + """' 
                and queryText like any(
                '%select%from%PRD3_1_010_DB_DWH%',
                '%select%from%PRD3_1_010_DB_STG%',
                '%select%from%PRD3_1_010_DB_TMD%',
                '%select%from%PRD3_1_010_VD_DWH%',
                '%select%from%PRD3_1_010_VD_TMD%',
                '%select%from%PRD3_1_015_DB_DWH%',
                '%select%from%PRD3_1_015_DB_STG%',
                '%select%from%PRD3_1_015_DB_TMD%',
                '%select%from%PRD3_1_015_VD_DWH%',
                '%select%from%PRD3_1_015_VD_TMD%',
                '%select%from%PRD3_1_016_DB_DWH%',
                '%select%from%PRD3_1_016_DB_STG%',
                '%select%from%PRD3_1_016_DB_TMD%',
                '%select%from%PRD3_1_016_VD_DWH%',
                '%select%from%PRD3_1_016_VD_TMD%',
                '%select%from%PRD3_1_029_DB_DWH%',
                '%select%from%PRD3_1_029_DB_STG%',
                '%select%from%PRD3_1_029_DB_TMD%',
                '%select%from%PRD3_1_029_VD_DWH%',
                '%select%from%PRD3_1_029_VD_TMD%',
                '%select%from%PRD3_1_037_DB_DWH%',
                '%select%from%PRD3_1_037_DB_STG%',
                '%select%from%PRD3_1_037_DB_TMD%',
                '%select%from%PRD3_1_037_VD_DWH%',
                '%select%from%PRD3_1_037_VD_TMD%',
                '%select%from%PRD3_1_068_DB_DWH%',
                '%select%from%PRD3_1_068_DB_STG%',
                '%select%from%PRD3_1_068_DB_TMD%',
                '%select%from%PRD3_1_068_VD_DWH%',
                '%select%from%PRD3_1_068_VD_TMD%',
                '%select%from%PRD3_1_069_DB_DWH%',
                '%select%from%PRD3_1_069_DB_STG%',
                '%select%from%PRD3_1_069_DB_TMD%',
                '%select%from%PRD3_1_069_VD_DWH%',
                '%select%from%PRD3_1_069_VD_TMD%',
                '%select%from%PRD3_1_076_DB_DWH%',
                '%select%from%PRD3_1_076_DB_STG%',
                '%select%from%PRD3_1_076_DB_TMD%',
                '%select%from%PRD3_1_076_VD_DWH%',
                '%select%from%PRD3_1_076_VD_TMD%',
                '%select%from%PRD3_1_083_DB_DWH%',
                '%select%from%PRD3_1_083_DB_STG%',
                '%select%from%PRD3_1_083_DB_TMD%',
                '%select%from%PRD3_1_083_VD_DWH%',
                '%select%from%PRD3_1_083_VD_TMD%',
                '%select%from%prd3_1_200_DB_DWH%',
                '%select%from%prd3_1_200_DB_STG%',
                '%select%from%prd3_1_200_DB_TMD%',
                '%select%from%prd3_1_200_VD_DWH%',
                '%select%from%prd3_1_200_VD_TMD%',
                '%select%from%PRD3_1_207_DB_DWH%',
                '%select%from%PRD3_1_207_DB_STG%',
                '%select%from%PRD3_1_207_DB_TMD%',
                '%select%from%PRD3_1_207_VD_DWH%',
                '%select%from%PRD3_1_207_VD_TMD%',
                '%select%from%PRD3_1_210_DB_DWH%',
                '%select%from%PRD3_1_210_DB_STG%',
                '%select%from%PRD3_1_210_DB_TMD%',
                '%select%from%PRD3_1_210_VD_DWH%',
                '%select%from%PRD3_1_210_VD_TMD%',
                '%select%from%PRD3_1_DB_DWH%',
                '%select%from%PRD3_1_DB_STG%',
                '%select%from%PRD3_1_DB_STG_DDL%',
                '%select%from%PRD3_1_DB_TMD_ARCH%',
                '%select%from%PRD3_1_VD_DWH%',
                '%select%from%PRD3_1_VD_TMD%',
                '%select%from%PRD3_DB_DWH%',
                '%select%from%PRD3_DB_STG%',
                '%select%from%PRD3_DB_STG_DDL%',
                '%select%from%PRD3_VD_DWH%',
                '%select%from%PRD3_VD_TMD%',
                '%select%from%PRD3_1_DB_DMOKFLEKP%',
                '%select%from%prd3_1_DB_DMPLACC%',
                '%select%from%PRD3_1_DB_DMPSSLCL%',
                '%select%from%PRD3_1_DB_DMSBCERTS%',
                '%select%from%prd3_1_DB_DMSLCL%',
                '%select%from%PRD3_1_DB_DMSPCGBK%',
                '%select%from%PRD3_1_DB_DMSPCPFL%',
                '%select%from%PRD3_1_DB_DMSPCRFPL%',
                '%select%from%PRD3_1_DB_DMTRANS%',
                '%select%from%PRD3_1_VD_DMOKFLEKP%',
                '%select%from%PRD3_1_VD_DMPLACC%',
                '%select%from%PRD3_1_VD_DMPSSLCL%',
                '%select%from%PRD3_1_VD_DMSBCERTS%',
                '%select%from%prd3_1_VD_DMSLCL%',
                '%select%from%PRD3_1_VD_DMSPCGBK%',
                '%select%from%PRD3_1_VD_DMSPCPFL%',
                '%select%from%PRD3_1_VD_DMSPCRFPL%',
                '%select%from%PRD3_1_VD_DMTRANS%',
                '%select%from%PRD3_vd_dwh_pr2%',
                '%select%from%PRD3_VD_PR%',
                '%select%from%PRD3_VD_PR2%')
                and statementType in ('select','insert','create table', 'update', 'merge into')
           
                group by 1,2,3,4,5
                """
        return SQLq
    
    #and username in (""" + SQLQuery_DM_users + """)
    #and username not in (""" + SQLQuery_all_users + """)
    #print('Main Stat: ', dataCreate(date_list[0], date_list[-1], 'Stat'))
    #print('Support Stat: ', dataCreate(date_list[0], date_list[-1], 'Stat_MDM'))
    
    if opt == 'Stat':
    
        for Date in date_list:
            print('Main Stat: ', Date, 'starts at ', datetime.now().strftime("%H:%M:%S"))
            sqlStr_main = dataCreate(Date, Date, opt)
            rows = TeraDB.execute_query(sqlStr_main)
            for row in rows:
                data_main.append(row)
    elif opt == 'Stat_MDM':
        for Date in date_list:
            print('Supported Stat: ', Date, 'starts at ', datetime.now().strftime("%H:%M:%S"))
            sqlStr_sup = dataCreate(Date, Date, opt)
            rows = TeraDB.execute_query(sqlStr_sup)
            for row in rows:
                data_sup.append(row)
    else:
        for Date in date_list:
            print('Main Stat: ', Date, 'starts at ', datetime.now().strftime("%H:%M:%S"))
            sqlStr_main = dataCreate(Date, Date, 'Stat')
            rows = TeraDB.execute_query(sqlStr_main)
            for row in rows:
                data_main.append(row)

            print('Supported Stat: ', Date, 'starts at ', datetime.now().strftime("%H:%M:%S"))
            sqlStr_sup = dataCreate(Date, Date, 'Stat_MDM')
            rows = TeraDB.execute_query(sqlStr_sup)
            for row in rows:
                data_sup.append(row)
        
    
    print('Calculation is completed: ', datetime.now().strftime("%H:%M:%S"))
    
    
    
    TeraDB.close_conn()
    
    df_sup = pd.DataFrame.from_records(data_sup, columns=['Date', 'Type', 'Item', 'User', 'DM', 'ImpactDisk', 'ReqPhysIOKB',
                                                          'AMPCPUTime',
                                                          'TotalIOcount', 'ImpactCPU', 'SQLcnt'])
    
    df_main = pd.DataFrame.from_records(data_main, columns=['Date', 'Type', 'Item', 'User', 'BB', 'ImpactDisk', 'ReqPhysIOKB',
                                                 'AMPCPUTime',
                                                 'TotalIOcount', 'ImpactCPU', 'SQLcnt'])
    
    end_date, end_time, end_tm = date.today(), time.localtime(), datetime.now()
    
    start_time = time.strftime("%H:%M:%S", start_time)
    end_time = time.strftime("%H:%M:%S", end_time)
    delta_tm = end_tm - start_tm
    dur_min = delta_tm.total_seconds() / 60
    
    print('Success for dates from {} to {}\nDuration is {} minutes'.format(start_dt, end_dt, dur_min))
    print('Running from ', start_date.strftime("%Y-%m-%d"), ' to ', end_date.strftime("%Y-%m-%d"))
    
    pivot_main = pd.pivot_table(df_main, values='AMPCPUTime', index='Item', columns='BB', aggfunc=np.sum, fill_value=0)
    pivot_sub = pd.pivot_table(df_sup, values='AMPCPUTime', index='DM', columns='Item', aggfunc=np.sum, fill_value=0)
    
    if export:
        with pd.ExcelWriter('/Desktop/Stat_Collect.xlsx') as writer:
            df_main.to_excel(writer, index = False, sheet_name = 'Data_Stat_BB')
            pivot_main.to_excel(writer, sheet_name = 'Pivot_Stat_BB')
            df_sup.to_excel(writer, index=False, sheet_name='Data_Stat_MDM')
            pivot_sub.to_excel(writer, sheet_name='Pivot_Stat_MDM')
    


collect_stat(start_dt, end_dt, Data, Save_file)

Start at  14:30:59
Main Stat:  2021-07-01 starts at  14:31:02
Main Stat:  2021-07-02 starts at  14:31:49
Main Stat:  2021-07-03 starts at  14:32:18
Main Stat:  2021-07-04 starts at  14:32:50
Main Stat:  2021-07-05 starts at  14:33:09
Main Stat:  2021-07-06 starts at  14:33:45
Main Stat:  2021-07-07 starts at  14:34:11
Main Stat:  2021-07-08 starts at  14:34:41
Main Stat:  2021-07-09 starts at  14:35:15
Main Stat:  2021-07-10 starts at  14:36:02
Main Stat:  2021-07-11 starts at  14:36:32
Main Stat:  2021-07-12 starts at  14:37:02
Main Stat:  2021-07-13 starts at  14:37:30
Main Stat:  2021-07-14 starts at  14:37:55
Main Stat:  2021-07-15 starts at  14:38:58
Main Stat:  2021-07-16 starts at  14:39:24
Main Stat:  2021-07-17 starts at  14:39:55
Main Stat:  2021-07-18 starts at  14:40:13
Main Stat:  2021-07-19 starts at  14:41:05
Main Stat:  2021-07-20 starts at  14:41:38
Main Stat:  2021-07-21 starts at  14:42:04
Main Stat:  2021-07-22 starts at  14:42:45
Main Stat:  2021-07-23 starts at  1